# Exportação de lista de Repescagem
Objetivo: Esse pipeline tem como objetivo separar a lista de repescagem de uma campanha, isto é, leads tipo 1 que não possuem conversão tipo 3 na campanha.

### Step 0: Importar dependências

In [1]:
# Importações básicas
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import os
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Adiciona src ao path
sys.path.append('../src')

#

# Utilitários de dados
from data_utils import (
    load_raw_data,
    save_processed_data,
    remove_duplicates,
    handle_missing_values,
    detect_outliers,
    normalize_column,
    process_phone_string,
    process_phone_number,
    clean_and_lower_column,
    flatten_list_to_df,
    remove_buyers_from_dataframe
)

CRONOGRAMA_SUBDOMAIN = 'cronogramadosfluentes-xwamel'

# Utilitários SQL
from sql_utils import DatabaseConnection as Dbc, load_query_from_file

# Utilitários de visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Utilitários de API
from api_utils import (
    make_request,
    get_json,
    post_json,
    paginated_request,
    response_to_dataframe
)

# utilitários hotmart
from hotmart_utils import Hotmart

# utilitários tmb
from tmb_utils import TMB   

# Configurações pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Load Database Driver
db = Dbc()

# Inicializar API Hotmart
hotmart = Hotmart()

# Inicializar API TMB
tmb = TMB()

print('✓ Importações concluídas com sucesso!')

✓ Importações concluídas com sucesso!


### Step 1: Load Dataframes

In [2]:
CAMPAIGN_ID = 'lcto-ofan-jan26'
MAIN_CONVERSION_TYPE_ID = 1
EXCLUSION_CONVERSION_TYPE_ID = '3'

# Load Leads Dataframe
df = db.execute_query_from_file("select_complete_lead_data_from_campaign", params={"campaign_id": CAMPAIGN_ID, "conversion_type_id": MAIN_CONVERSION_TYPE_ID})

# Load Exclusion Leads Dataframe
#exclusion_df = db.execute_query_from_file("select_complete_lead_data_from_campaign", params={"campaign_id": CAMPAIGN_ID, "conversion_type_id": EXCLUSION_CONVERSION_TYPE_ID})
exclusion_df = load_raw_data('ofan_leads.csv',sep=';')

### Step 2: Exclude leads

In [8]:
df['phone'] = process_phone_string(df, 'phone')
df['whatsapp_format'] = process_phone_string(df, 'whatsapp_format')

df_final = df[~df['phone'].isin(exclusion_df['Nome'].astype(str).unique())]
df_final = df[~df['whatsapp_format'].isin(exclusion_df['Nome'].astype(str).unique())]


print(df.shape  )
print(df_final.shape)

(10525, 19)
(3050, 19)
